<a href="https://colab.research.google.com/github/cod3astro/ML_practice/blob/main/codon_usage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [240]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

In [241]:
df = pd.read_csv('codon_usage.csv', index_col='SpeciesID')
df.drop_duplicates(inplace=True)
df.tail(40)

,Kingdom,DNAtype,Ncodons,SpeciesName,UUU,UUC,UUA,UUG,CUU,CUC,...,CGG,AGA,AGG,GAU,GAC,GAA,GAG,UAA,UAG,UGA
SpeciesID,,,,,,,,,,,,,,,,,,,,,
9521,pri,1,4560,mitochondrion Saimiri sciureus,0.02895,0.03421,0.02588,0.00219,0.02675,0.04167,...,0.00044,0.00000,0.00000,0.00482,0.01886,0.01535,0.00044,0.00263,0.00000,0.02412
9523,pri,0,1815,Callicebus moloch,0.02314,0.04022,0.00331,0.01047,0.01212,0.03471,...,0.01102,0.00331,0.01653,0.01157,0.02920,0.02094,0.03471,0.00000,0.00165,0.00275
9530,pri,0,2817,Cercocebus torquatus,0.01775,0.02556,0.00603,0.01526,0.01349,0.02556,...,0.00745,0.01278,0.01384,0.02236,0.02804,0.03088,0.03550,0.00177,0.00106,0.00213
9531,pri,0,9371,Cercocebus torquatus atys,0.02273,0.02924,0.00768,0.01451,0.01505,0.02273,...,0.00534,0.01441,0.01419,0.01654,0.01782,0.02518,0.03319,0.00096,0.00021,0.00203
9534,pri,0,73417,Chlorocebus aethiops,0.01839,0.02272,0.00689,0.01238,0.01531,0.01921,...,0.00929,0.01282,0.01047,0.02107,0.02223,0.02702,0.03582,0.00048,0.00054,0.00121
9534,pri,1,2289,mitochondrion Chlorocebus aethiops,0.02228,0.02578,0.02315,0.00393,0.01791,0.03495,...,0.00131,0.00000,0.00044,0.00568,0.01092,0.02010,0.00612,0.00175,0.00175,0.01616
9538,pri,0,2903,Erythrocebus patas,0.01791,0.03789,0.00448,0.00930,0.01033,0.03651,...,0.01309,0.01171,0.00930,0.01240,0.01826,0.01585,0.03617,0.00034,0.00034,0.00241
9540,pri,0,2089,Macaca arctoides,0.02106,0.02824,0.00766,0.01197,0.01484,0.03064,...,0.00766,0.01101,0.01340,0.01101,0.02058,0.01915,0.02346,0.00048,0.00048,0.00191
9541,pri,0,2451969,Macaca fascicularis,0.02035,0.0188,0.00952,0.01460,0.01524,0.01765,...,0.00994,0.01428,0.01206,0.02414,0.02299,0.03325,0.03704,0.00118,0.00082,0.00173


In [242]:
df.shape

(13028, 68)

In [243]:
df.columns

Index(['Kingdom', 'DNAtype', 'Ncodons', 'SpeciesName', 'UUU', 'UUC', 'UUA',
       'UUG', 'CUU', 'CUC', 'CUA', 'CUG', 'AUU', 'AUC', 'AUA', 'AUG', 'GUU',
       'GUC', 'GUA', 'GUG', 'GCU', 'GCC', 'GCA', 'GCG', 'CCU', 'CCC', 'CCA',
       'CCG', 'UGG', 'GGU', 'GGC', 'GGA', 'GGG', 'UCU', 'UCC', 'UCA', 'UCG',
       'AGU', 'AGC', 'ACU', 'ACC', 'ACA', 'ACG', 'UAU', 'UAC', 'CAA', 'CAG',
       'AAU', 'AAC', 'UGU', 'UGC', 'CAU', 'CAC', 'AAA', 'AAG', 'CGU', 'CGC',
       'CGA', 'CGG', 'AGA', 'AGG', 'GAU', 'GAC', 'GAA', 'GAG', 'UAA', 'UAG',
       'UGA'],
      dtype='object')

In [244]:
df.isnull().any().sum()

np.int64(0)

In [245]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.feature_selection import mutual_info_classif
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, confusion_matrix
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from imblearn.over_sampling import SMOTE

In [246]:
features = ['DNAtype', 'UUA',
       'UUG', 'CUU', 'CUC', 'CUA', 'CUG', 'AUU', 'AUC', 'AUA', 'AUG', 'GUU',
       'GUC', 'GUA', 'GUG', 'GCU', 'GCC', 'GCA', 'GCG', 'CCU', 'CCC', 'CCA',
       'CCG', 'UGG', 'GGU', 'GGC', 'GGA', 'GGG', 'UCU', 'UCC', 'UCA', 'UCG',
       'AGU', 'AGC', 'ACU', 'ACC', 'ACA', 'ACG', 'UAU', 'UAC', 'CAA', 'CAG',
       'AAU', 'AAC', 'UGU', 'UGC', 'CAU', 'CAC', 'AAA', 'AAG', 'CGU', 'CGC',
       'CGA', 'CGG', 'AGA', 'AGG', 'GAU', 'GAC', 'GAA', 'GAG', 'UAA', 'UAG',
       'UGA']
mi_features = ['DNAtype', 'Ncodons', 'UUU', 'UUC']
target = 'Kingdom'
X = df[features]
y = df[target]
smote = SMOTE(random_state=42)
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=42, stratify=y, test_size=0.2)

In [247]:
for col in X:
  if X[col].dtype == 'object':
    print(col)

In [248]:
for col in X.columns:
    if X[col].dtype == 'object':
        # Find rows with non-numeric values
        non_numeric = X[col][~X[col].apply(lambda x: str(x).replace(';', '').isdigit())]
        if not non_numeric.empty:
            print(f"Column '{col}' has non-numeric values:")
            print(non_numeric.unique())

In [249]:
for col in X.columns:
    # Check if any value is a string that can't convert to float
    non_float = []
    for val in X[col]:
        try:
            float(val)
        except (ValueError, TypeError):
            non_float.append(val)

    if non_float:
        print(f"Column '{col}' has non-float values:")
        print(pd.Series(non_float).unique())

In [250]:
print("train_X dtypes:", train_X.dtypes)
print("val_X dtypes:", val_X.dtypes)

train_X dtypes: DNAtype      int64
UUA        float64
UUG        float64
CUU        float64
CUC        float64
            ...   
GAA        float64
GAG        float64
UAA        float64
UAG        float64
UGA        float64
Length: 63, dtype: object
val_X dtypes: DNAtype      int64
UUA        float64
UUG        float64
CUU        float64
CUC        float64
            ...   
GAA        float64
GAG        float64
UAA        float64
UAG        float64
UGA        float64
Length: 63, dtype: object


In [251]:
copy_X = X.copy()
copy_y = y.copy()
for col in copy_X.select_dtypes('float'):
    copy_X[col] = copy_X[col].round().astype(int)
for char in copy_X.select_dtypes('object'):
  copy_X.loc[:, char], _ = copy_X[char].factorize()
discrete_values = copy_X.dtypes == int
copy_y, _ = pd.factorize(y)

def make_mi_score(copy_X, copy_y, discrete_values):
    mi_scores = mutual_info_classif(copy_X, copy_y, discrete_features=discrete_values)
    mi_scores = pd.Series(mi_scores, name='MI SCORES', index=copy_X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

mi_scores = make_mi_score(copy_X, copy_y, discrete_values)
mi_scores[:7]

,MI SCORES
DNAtype,0.403521
UUA,0.000000
UUG,0.000000
CUU,0.000000
CUC,0.000000
CUA,0.000000
CUG,0.000000


In [252]:
y.value_counts(normalize=True)

,proportion
Kingdom,
bct,0.224133
vrl,0.217378
pln,0.193660
vrt,0.159426
inv,0.103239
mam,0.043905
phg,0.016887
rod,0.016503
pri,0.013816


In [253]:
X_resampled, y_resampled = smote.fit_resample(X, y)
X_train, X_val, y_train, y_val = train_test_split(X_resampled, y_resampled, random_state=42, stratify=y_resampled, test_size=0.2)

In [254]:
model = Pipeline([
    ('model', XGBClassifier(random_state=42, class_weight='balanced', eval_metric='mlogloss', n_estimators=300, max_depth=6, learning_rate=0.14))
])
label_encoder = LabelEncoder()
train_y_encoded = label_encoder.fit_transform(train_y)
val_y_encoded = label_encoder.transform(val_y)

model.fit(train_X, train_y_encoded)
xgb_train_preds_encoded = model.predict(val_X)
xgb_train_preds = label_encoder.inverse_transform(xgb_train_preds_encoded)
accuracy_score(val_y, xgb_train_preds)

0.930928626247122

In [255]:
model = Pipeline([
    ('model', XGBClassifier(random_state=42, eval_metric='mlogloss', n_estimators=300, max_depth=7, learning_rate=0.13))
])
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)

model.fit(X_train, y_train_encoded)
xgb_train_preds_encoded = model.predict(X_val)
xgb_train_preds = label_encoder.inverse_transform(xgb_train_preds_encoded)
accuracy_score(y_val, xgb_train_preds)

0.9834993773349938

In [256]:
model = Pipeline([
    ('model', RandomForestClassifier(random_state=42, class_weight='balanced', n_estimators=300, max_depth=30))
])
label_encoder = LabelEncoder()
train_y_encoded = label_encoder.fit_transform(train_y)
val_y_encoded = label_encoder.transform(val_y)

model.fit(train_X, train_y_encoded)
rf_train_preds_encoded = model.predict(val_X)
rf_train_preds = label_encoder.inverse_transform(rf_train_preds_encoded)
accuracy_score(val_y, rf_train_preds)

0.9071373752877974

In [257]:
model = Pipeline([
    ('model', RandomForestClassifier(random_state=42, n_estimators=300, max_depth=20))
])
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)

model.fit(X_train, y_train_encoded)
rf_train_preds_encoded = model.predict(X_val)
rf_train_preds = label_encoder.inverse_transform(rf_train_preds_encoded)
accuracy_score(y_val, rf_train_preds)

0.9796077210460772

In [258]:
scores = cross_val_score(
    RandomForestClassifier(),
    X[['DNAtype']].apply(lambda x: pd.factorize(x)[0]),  # Convert to numeric
    y,
    cv=5
)
print(f"Accuracy using only DNAtype: {scores.mean():.2f}")

Accuracy using only DNAtype: 0.41
